In [1]:
import os

import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

In [3]:
app = Flask(__name__)
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///TeamPlaisant/aid_data.sqlite"
db = SQLAlchemy(app)

# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(db.engine, reflect=True)

aid = Base.classes.aid_data

C:\Users\llreb\Anaconda3\envs\PythonData\lib\site-packages\flask_sqlalchemy\__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [4]:
year = 2016

In [7]:
sel = [
        aid.country_name,
        aid.latitude,
        aid.longitude,
        aid.transaction_type_name,
        func.sum(aid.constant_amount),
        aid.fiscal_year        
    ]

results = db.session.query(*sel).filter(aid.fiscal_year == year).filter(aid.transaction_type_name == "Disbursements").group_by(aid.country_name, aid.fiscal_year, aid.transaction_type_name)

# Create a dictionary entry for each row of information

map_data = []
for result in results:
    aid_dict = {}
    aid_dict["country"] = result[0]
    aid_dict["latitude"] = result[1]
    aid_dict["longitude"] = result[2]
    aid_dict["transaction type"] = result[3]
    aid_dict["amount"] = result[4]
    aid_dict["year"] = result[5]
    map_data.append(aid_dict)
print(map_data)

[{'country': 'Afghanistan', 'latitude': 33.93911, 'longitude': 67.709953, 'transaction type': 'Disbursements', 'amount': 4223684237, 'year': 2016}, {'country': 'Albania', 'latitude': 41.153332, 'longitude': 20.168331, 'transaction type': 'Disbursements', 'amount': 28703594, 'year': 2016}, {'country': 'Algeria', 'latitude': 28.033886, 'longitude': 1.6596259999999998, 'transaction type': 'Disbursements', 'amount': 8341056, 'year': 2016}, {'country': 'Angola', 'latitude': -11.202691999999999, 'longitude': 17.873887, 'transaction type': 'Disbursements', 'amount': 69153127, 'year': 2016}, {'country': 'Antigua and Barbuda', 'latitude': 17.060816, 'longitude': -61.796428000000006, 'transaction type': 'Disbursements', 'amount': 635781, 'year': 2016}, {'country': 'Argentina', 'latitude': -38.416097, 'longitude': -63.616671999999994, 'transaction type': 'Disbursements', 'amount': 6148126, 'year': 2016}, {'country': 'Armenia', 'latitude': 40.069099, 'longitude': 45.038189, 'transaction type': 'Di

In [4]:
stmt = db.session.query(aid).statement
df = pd.read_sql_query(stmt, db.session.bind)
df.head()

,index,country_code,country_name,latitude,longitude,region_name,income_group_name,income_group_acronym,implementing_agency_acronym,implementing_agency_name,...,activity_project_number,activity_start_date,activity_end_date,transaction_type_id,transaction_type_name,fiscal_year,current_amount,constant_amount,USG_sector_name,submission_id
0,0,AFG,Afghanistan,33.93911,67.709953,South and Central Asia,Low Income Country,LIC,DOD,Department of Defense,...,None,None,None,2,Obligations,2011,9941000000,10731991839,Stabilization Operations and Security Sector R...,28
1,1,AFG,Afghanistan,33.93911,67.709953,South and Central Asia,Low Income Country,LIC,DOD,Department of Defense,...,None,None,None,2,Obligations,2012,9243000000,9799467226,Stabilization Operations and Security Sector R...,28
2,2,AFG,Afghanistan,33.93911,67.709953,South and Central Asia,Low Income Country,LIC,DOD,Department of Defense,...,None,None,None,3,Disbursements,2011,7840175215,8464007285,Stabilization Operations and Security Sector R...,28
3,3,AFG,Afghanistan,33.93911,67.709953,South and Central Asia,Low Income Country,LIC,DOD,Department of Defense,...,None,None,None,3,Disbursements,2013,7764310985,8095048196,Stabilization Operations and Security Sector R...,28
4,4,AFG,Afghanistan,33.93911,67.709953,South and Central Asia,Low Income Country,LIC,DOD,Department of Defense,...,None,None,None,2,Obligations,2013,6928000000,7223112779,Stabilization Operations and Security Sector R...,28


In [5]:
# Filter the data based on the year
year = 2016

aid_data = df.loc[(df["fiscal_year"] == year) & (df["transaction_type_name"] == "Disbursements"), ["country_name", "latitude", "longitude", "transaction_type_name", "constant_amount", "fiscal_year"]]
# Format the data to send as json
aid_data.head()



,country_name,latitude,longitude,transaction_type_name,constant_amount,fiscal_year
34,Israel,31.046051,34.851612,Disbursements,3100000000,2016
47,Iraq,33.223191,43.679291,Disbursements,2850000000,2016
56,Afghanistan,33.939110,67.709953,Disbursements,2601000000,2016
89,World,NaN,NaN,Disbursements,1818282317,2016
213,World,NaN,NaN,Disbursements,1197128000,2016


In [26]:
aid_by_country = aid_data.groupby(["country_name", "latitude", "longitude", "fiscal_year", "transaction_type_name"])["constant_amount"].sum()
aid_by_country_df = pd.DataFrame(aid_by_country).set_index("country_name")
aid_by_country_df.head()
#aid_by_country

KeyError: 'country_name'

In [24]:
map_data = {"country": aid_by_country_df.country_name.values.to_list, 
                    "year": aid_by_country_df.fiscal_year.values.to_list,
                    "latitude": aid_by_country_df.latitude.values.to_list, 
                     "longitue": aid_by_country_df.longitude.values.to_list,
                     "amount": aid_by_country_df.constant_amount.values.to_list}


AttributeError: 'DataFrame' object has no attribute 'country_name'

In [20]:
map_data = list(np.ravel(aid_by_country))
map_data

[4223684237,
 28703594,
 8341056,
 69153127,
 635781,
 6148126,
 46024007,
 2538,
 71269,
 24501071,
 2694794,
 7149660,
 266099370,
 4244751,
 10646578,
 7073320,
 67166776,
 356477,
 1684111,
 46034367,
 75565717,
 21083520,
 6149,
 327814,
 18159686,
 64543906,
 388739854,
 107886028,
 100007980,
 21081443,
 70365720,
 71720603,
 2269112,
 414951466,
 1354220,
 12761735,
 3673996,
 11978547,
 1620405,
 3455,
 8580761,
 765392,
 71633553,
 19440433,
 1281256718,
 90125230,
 911439,
 136488,
 16013027,
 967145636,
 3922098,
 104419,
 2808778,
 2487585,
 130716188,
 857807,
 212131676,
 5049805,
 622300,
 177049368,
 134519213,
 9853132,
 7714041,
 435084745,
 152127947,
 11829507,
 123091426,
 287075232,
 1067832,
 4256991847,
 3116297335,
 23648752,
 103284,
 1227925177,
 74820978,
 893457494,
 65631597,
 128171,
 59348825,
 45772055,
 20856967,
 254886322,
 39832867,
 437851172,
 17217485,
 15713051,
 5242,
 101833766,
 379408277,
 10837700,
 2444977,
 186672008,
 506573,
 79665613,